In [1]:
import os

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['Vehicle Page'] = input_['Vehicle Page'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：550


['BMP02A',
 'BMP03A',
 'BMP05A',
 'BMP06A',
 'BMP08A',
 'BMP09A',
 'BMP10A',
 'BMP11A',
 'BMP13A',
 'BMP14A',
 'BMP15A',
 'BMP17A',
 'BMP19A',
 'BMP20A',
 'BMP21A',
 'BMP22A',
 'BMP23A',
 'BMP24A',
 'CATP04A',
 'CATP05A',
 'CMP01A',
 'CMP02A',
 'CMP03A',
 'CMP04A',
 'CMP05A',
 'CMP06A',
 'CMP08A',
 'CMP09A',
 'CRP01A',
 'CRP02A',
 'CRP03A',
 'CRP03B',
 'CRP04B',
 'CRP05A',
 'CRP05B',
 'CRP06A',
 'CRP06B',
 'CRP06C',
 'CRP07A',
 'CRP08A',
 'CRP09B',
 'CRP10A',
 'CRP11A',
 'CRP12A',
 'CRP14A',
 'CRP14B',
 'CRP15A',
 'CRP16A',
 'CRP18A',
 'CRP19A',
 'CRP19B',
 'CRP25A',
 'CRP26A',
 'CRP28A',
 'CRP30A',
 'CRP31A',
 'CRP32A',
 'CRP33A',
 'CRP33B',
 'CRP34A',
 'CRP35A',
 'CRP35B',
 'CRP37A',
 'CRP38A',
 'CRP39A',
 'CRP40A',
 'CRP41A',
 'CRP42A',
 'CRP43A',
 'CRP44A',
 'CRP44B',
 'CRP45A',
 'CRP46A',
 'CRP47A',
 'CRP49A',
 'CRP50A',
 'CRP51A',
 'CRP52A',
 'CRP53A',
 'CRP54A',
 'CRP55A',
 'CRP56A',
 'CRP57A',
 'CRP58A',
 'CRP59A',
 'CRP60A',
 'CRP61A',
 'CRP62A',
 'CRP63A',
 'CRP64A',
 'CRP65A

In [5]:
a = 1

a

1

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

df_temp, h3_last, page = pd.DataFrame(), '', 1
while page <= input_.loc[a, 'Vehicle Page']:
    b = 0
    while True:
        b += 1
        print(b, page)
        try:
            request_url = f'''{input_.loc[a, 'Url']}?page={page}#applications'''
            
            resp = requests.get(request_url,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))
                
                if html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip() == input_.loc[a, 'Part Number']:
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_h3, list_table = [text.strip() for text in html.xpath('//section[@id="applications"]/h3[@class="part__titre-groupe-vehicules"]/text()')], html.xpath('//section[@id="applications"]/table[@class="part__table-applications"]')

    # = = = = = = = = = = = = = = =

    if len(list_table) == 0:
        print('fuck')

    # = = = = = = = = = = = = = = =

    if len(list_h3) == len(list_table):
        for (i, h3), table in zip(enumerate(list_h3), list_table):
            list_th, list_row = [th.xpath('./text()')[0].strip() for th in table.xpath('./thead/tr/th')[1:]], [[td.xpath('./text()')[0].strip() for td in tr.xpath('./td[@class="part__table-applications__data"]')] for tr in table.xpath('./tbody/tr')]
            df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Page': page,
                                         'Vehicle No': i + 1,
                                         'Make Model Engine': h3,
                                         'Row': [j+1 for j in range(len(list_row))]})

            for j, row in enumerate(list_row):
                for k, th in enumerate(list_th):
                    df_temp_temp.loc[j, th] = row[k]

            df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
    elif len(list_h3) + 1 == len(list_table):
        list_th, list_row = [th.xpath('./text()')[0].strip() for th in list_table[0].xpath('./thead/tr/th')[1:]], [[td.xpath('./text()')[0].strip() for td in tr.xpath('./td[@class="part__table-applications__data"]')] for tr in list_table[0].xpath('./tbody/tr')]
        df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                     'Part Number': input_.loc[a, 'Part Number'],
                                     'Page': page,
                                     'Vehicle No': -1,
                                     'Make Model Engine': h3_last,
                                     'Row': [j+1 for j in range(len(list_row))]})

        for j, row in enumerate(list_row):
            for k, th in enumerate(list_th):
                df_temp_temp.loc[j, th] = row[k]
                
        df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
        
        list_table.pop(0)
        
        for (i, h3), table in zip(enumerate(list_h3), list_table):
            list_th, list_row = [th.xpath('./text()')[0].strip() for th in table.xpath('./thead/tr/th')[1:]], [[td.xpath('./text()')[0].strip() for td in tr.xpath('./td[@class="part__table-applications__data"]')] for tr in table.xpath('./tbody/tr')]
            df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Page': page,
                                         'Vehicle No': i + 1,
                                         'Make Model Engine': h3,
                                         'Row': [j+1 for j in range(len(list_row))]})

            for j, row in enumerate(list_row):
                for k, th in enumerate(list_th):
                    df_temp_temp.loc[j, th] = row[k]

            df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
    else:
        print('fuck')

    # = = = = = = = = = = = = = = =

    h3_last = list_h3[-1]

    page += 1

    # = = = = = = = = = = = = = = =

df_temp

1 1


,No,Part Number,Page,Vehicle No,Make Model Engine,Row,Year,Fuel,Fuel Del.,ASP.,Eng. VIN,Eng. Desg.,CC,CID
0,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,1,2005,GAS,FI,N,-,M54,2494,152
1,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,2,2004,GAS,FI,N,-,M54,2494,152
2,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,3,2003,GAS,FI,N,-,M54,2494,152
3,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,4,2002,GAS,FI,N,-,M54,2494,152
4,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,5,2001,GAS,FI,N,-,M54,2494,152
5,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,1,2005,GAS,FI,N,-,M54,2979,-
6,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,2,2004,GAS,FI,N,-,M54,2979,-
7,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,3,2003,GAS,FI,N,-,M54,2979,-
8,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,4,2002,GAS,FI,N,-,M54,2979,-
9,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,5,2001,GAS,FI,N,-,M54,2979,-


In [8]:
df_temp.empty

False

In [9]:
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,Page,Vehicle No,Make Model Engine,Row,Year,Fuel,Fuel Del.,ASP.,Eng. VIN,Eng. Desg.,CC,CID
0,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,1,2005,GAS,FI,N,-,M54,2494,152
1,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,2,2004,GAS,FI,N,-,M54,2494,152
2,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,3,2003,GAS,FI,N,-,M54,2494,152
3,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,4,2002,GAS,FI,N,-,M54,2494,152
4,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,5,2001,GAS,FI,N,-,M54,2494,152
5,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,1,2005,GAS,FI,N,-,M54,2979,-
6,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,2,2004,GAS,FI,N,-,M54,2979,-
7,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,3,2003,GAS,FI,N,-,M54,2979,-
8,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,4,2002,GAS,FI,N,-,M54,2979,-
9,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,5,2001,GAS,FI,N,-,M54,2979,-


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct['Year'] = output_correct['Year'].astype(int)
    
    output_correct = output_correct.sort_values(by=['No', 'Page', 'Vehicle No', 'Row'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Part Number,Page,Vehicle No,Make Model Engine,Row,Year,Fuel,Fuel Del.,ASP.,Eng. VIN,Eng. Desg.,CC,CID
0,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,1,2005,GAS,FI,N,-,M54,2494,152
1,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,2,2004,GAS,FI,N,-,M54,2494,152
2,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,3,2003,GAS,FI,N,-,M54,2494,152
3,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,4,2002,GAS,FI,N,-,M54,2494,152
4,2,BMP03A,1,1,BMW 325xi Base L6 2.5L,5,2001,GAS,FI,N,-,M54,2494,152
5,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,1,2005,GAS,FI,N,-,M54,2979,-
6,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,2,2004,GAS,FI,N,-,M54,2979,-
7,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,3,2003,GAS,FI,N,-,M54,2979,-
8,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,4,2002,GAS,FI,N,-,M54,2979,-
9,2,BMP03A,1,2,BMW 330xi Base L6 3.0L,5,2001,GAS,FI,N,-,M54,2979,-


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
